In [4]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("kaiyuy/leandojo-lean4-tacgen-byt5-small")
model = AutoModelForSeq2SeqLM.from_pretrained("kaiyuy/leandojo-lean4-tacgen-byt5-small")

state = "n : ℕ\n⊢ gcd n n = n"
tokenized_state = tokenizer(state, return_tensors="pt")

# Generate a single tactic.
tactic_ids = model.generate(tokenized_state.input_ids, max_length=1024)
tactic = tokenizer.decode(tactic_ids[0], skip_special_tokens=True)
print(tactic, end="\n\n")

# Generate multiple tactics via beam search.
tactic_candidates_ids = model.generate(
    tokenized_state.input_ids,
    max_length=1024,
    num_beams=4,
    length_penalty=0.0,
    do_sample=False,
    num_return_sequences=4,
    early_stopping=False,
)
tactic_candidates = tokenizer.batch_decode(
    tactic_candidates_ids, skip_special_tokens=True
)
for tac in tactic_candidates:
    print(tac)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


rw [<a>Nat.gcd_comm</a>]

rw [<a>Nat.gcd_comm</a>]
rw [<a>Nat.gcd_comm</a>, <a>Nat.gcd_1</a>]
rw [<a>Nat.gcd_comm</a>, <a>Nat.gcd_rec</a>]
rw [<a>Nat.gcd_comm</a>, <a>Nat.gcd_gcd_self_left_right</a>]


In [3]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("kaiyuy/leandojo-lean4-retriever-tacgen-byt5-small")
model = AutoModelForSeq2SeqLM.from_pretrained("kaiyuy/leandojo-lean4-retriever-tacgen-byt5-small")

state = "n : ℕ\n⊢ gcd n n = n"
retrieved_premises = [
  "def <a>Nat.gcd</a> : Nat → Nat → Nat\n| 0        y := y\n| (succ x) y := have y % succ x < succ x, from mod_lt _ $ succ_pos _,\n                gcd (y % succ x) (succ x)",
  "@[simp] theorem <a>Nat.mod_self</a> (n : Nat) : n % n = 0",
]
input = "\n\n".join(retrieved_premises + [state])
print("------ INPUT ------\n", input)
tokenized_input = tokenizer(input, return_tensors="pt", max_length=2300, truncation=True)

# Generate a single tactic.
tactic_ids = model.generate(tokenized_input.input_ids, max_length=1024)
tactic = tokenizer.decode(tactic_ids[0], skip_special_tokens=True)
print("\n------ OUTPUT ------")
print(tactic, end="\n\n")

# Generate multiple tactics via beam search.
tactic_candidates_ids = model.generate(
    tokenized_input.input_ids,
    max_length=1024,
    num_beams=4,
    length_penalty=0.0,
    do_sample=False,
    num_return_sequences=4,
    early_stopping=False,
)
tactic_candidates = tokenizer.batch_decode(
    tactic_candidates_ids, skip_special_tokens=True
)
for tac in tactic_candidates:
    print(tac)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


------ INPUT ------
 n : ℕ
⊢ gcd n n = n

------ OUTPUT ------
rw [<a>gcd_eq_nth_of_gcd_eq_one</a> n n (n + 1) = n : ℕ : ℕ → ℕ → <a>GCD</gcd</n n gcd_n n = n : ℕ : ℕ → ℕ → <a>GCD</gcd</n n = n := <a>EqGCD.gcd_eq_nth_of_n n (n + 1) n <;> simp [<a>gcd_eqgcd_n_neg_nth_eq_n n]

rw [<a>gcd_eq_nth_of_gcd_eq_one</a>, <a>gcd_eq_nth_of_nth_of_nth_of_nth_of_nth_of_nth_of_nth_of_nth_of_nth_of_nth_of_nth_of_nth_of_nth_of_nth_of_nth_of_nth_of_nth_of_nth_of_nth_of_nth_of_nth_of_nth_of_nth_of_nth_of_nth_of_nth_of_nth_of_nth_of_nth_of_nth_of_nth_of_nth_of_nth_of_nth_of_nth_of_nth_of_nth_of_nth_of_nth_of_nth_of_nth_of_nth_of_nth_of_nth_of_nth_of_nth_of_nth_of_nth_of_nth_of_nth_of_nth_of_nth_of_nth_of_nth_of_nth_of_nth_of_nth_of_nth_of_nth_of_nth_of_nth_of_nth_of_nth_of_nth_of_nth_of_nth_of_nth_of_nth_of_nth_of_nth_of_nth_of_nth_of_nth_of_nth_of_nth_of_nth_of_nth_of_nth_of_nth_of_nth_of_nth_of_nth_of_nth_of_nth_of_nth_of_nth_of_nth_of_nth_of_nth_of_nth_of_nth_of_nth_of_nth_of_nth_of_nth_of_nth_of_nth_of

In [ ]:

# Create indexed corpus on all premises (with their embeddings)
python retrieval/index.py \
--ckpt_path /home/mcwave/code/3rdparty/ReProver/ckpt/leandojo-pl-ckpts/retriver_novel_premises.ckpt \
--corpus-path /home/mcwave/code/3rdparty/ReProver/data/leandojo_benchmark_4/corpus.jsonl \
--output-path /home/mcwave/code/3rdparty/ReProver/data/indexed_corpus


# python retrieval/index.py \
# --ckpt_path /home/mcwave/code/3rdparty/ReProver/lightning_logs/version_2/checkpoints/last.ckpt \
# --corpus-path /home/mcwave/code/3rdparty/ReProver/data/leandojo_benchmark_4/corpus.jsonl \
# --output-path /home/mcwave/code/3rdparty/ReProver/data/indexed_corpus


python prover/evaluate.py \
--data-path /home/mcwave/code/3rdparty/ReProver/data/leandojo_benchmark_4/novel_premises/  \
--ckpt_path /home/mcwave/code/3rdparty/ReProver/ckpt/leandojo-pl-ckpts/reprover_novel_premises.ckpt \
--indexed-corpus-path /home/mcwave/code/3rdparty/ReProver/data/indexed_corpus \
--split test --num-workers 5 --num-gpus 1


In [ ]:
"""
Theorem Proving Evaluation on LeanDojo Benchmark
After the tactic generator is trained, we combine it with best-first search to prove theorems by interacting with Lean.

For models without retrieval, run:
"""

python prover/evaluate.py \
--data-path /home/mcwave/code/3rdparty/ReProver/data/leandojo_benchmark_4/novel_premises/ \
--ckpt_path /home/mcwave/code/3rdparty/ReProver/ckpt/leandojo-pl-ckpts/generator_novel_premises.ckpt \
--split test --num-workers 1 --num-gpus 1
